In [2]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import timeit
import os
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from utils import *

# Data

## Get Data

In [9]:
yield_rate = pd.read_excel("../data/yield.xlam")
yield_rate['Date'] = yield_rate['Date'].apply(yield_convert_time)
yield_rate = yield_rate[['Date', "2 Yr", "10 Yr"]]
yield_rate.head(), yield_rate.tail()

(       Date  2 Yr  10 Yr
 0  19900102  7.87   7.94
 1  19900103  7.94   7.99
 2  19900104  7.92   7.98
 3  19900105  7.90   7.99
 4  19900108  7.90   8.02,
           Date  2 Yr  10 Yr
 7716  20201102  0.16   0.87
 7717  20201103  0.17   0.90
 7718  20201104  0.14   0.78
 7719  20201105  0.14   0.79
 7720  20201106  0.16   0.83)

## get 5%, 10%, 20%, 30% percentiles

In [28]:
# difference between 2 Yr and 10 Yr
diff = np.array(yield_rate["10 Yr"]) - np.array(yield_rate["2 Yr"])
diff

array([0.07, 0.05, 0.06, ..., 0.64, 0.65, 0.67])

In [41]:
five_perc = np.nanpercentile(diff, 5, axis = 0)
ten_perc = np.nanpercentile(diff, 10, axis = 0)
twenty_perc = np.nanpercentile(diff, 20, axis = 0)
thirty_perc = np.nanpercentile(diff, 30, axis = 0)
five_perc, ten_perc, twenty_perc, thirty_perc

(-0.04999999999999982,
 0.08000000000000007,
 0.22999999999999998,
 0.4500000000000002)

## get corresponding dates

In [18]:
yield_rate.loc[yield_rate['Date'] == '20080215']

,Date,2 Yr,10 Yr
4534,20080215,1.91,3.76


In [24]:
yield_used = yield_rate.iloc[4534:].reset_index(drop = True)
yield_used.head()

,Date,2 Yr,10 Yr
0,20080215,1.91,3.76
1,20080219,2.06,3.89
2,20080220,2.14,3.93
3,20080221,1.98,3.77
4,20080222,1.98,3.79


# conversion function

In [47]:
# difference between 2 Yr and 10 Yr
diff = np.array(yield_used["10 Yr"]) - np.array(yield_used["2 Yr"])
# calculate percentiles
five_perc = np.nanpercentile(diff, 5, axis = 0)
ten_perc = np.nanpercentile(diff, 10, axis = 0)
twenty_perc = np.nanpercentile(diff, 20, axis = 0)
thirty_perc = np.nanpercentile(diff, 30, axis = 0)
five_perc, ten_perc, twenty_perc, thirty_perc

(0.17999999999999994, 0.2599999999999998, 0.5500000000000003, 0.95)

In [48]:
conversion_rate = []
for i in range(len(yield_used)):
    cur_yield = yield_used.iloc[i]
    cur_diff = cur_yield['10 Yr'] - cur_yield['2 Yr']
    print(cur_diff)
    if cur_diff <= five_perc:
        print("in five")
        conversion_rate.append(0.5)
    elif cur_diff <= ten_perc:
        print("in ten")
        conversion_rate.append(0.6)
    elif cur_diff <= twenty_perc:
        print("in twenty")
        conversion_rate.append(0.75)
    elif cur_diff <= thirty_perc:
        print("in thirty")
        conversion_rate.append(0.9)
    else:
        print("in else")
        conversion_rate.append(1.0)
len(conversion_rate), conversion_rate

1.8499999999999999
in else
1.83
in else
1.79
in else
1.79
in else
1.81
in else
1.7800000000000002
in else
1.8399999999999999
in else
1.8400000000000003
in else
1.8399999999999999
in else
1.88
in else
1.93
in else
1.98
in else
2.04
in else
2.09
in else
2.0300000000000002
in else
1.99
in else
1.86
in else
1.8600000000000003
in else
1.9300000000000002
in else
1.97
in else
1.9899999999999998
in else
1.9
in else
1.8399999999999999
in else
1.7499999999999998
in else
1.72
in else
1.7199999999999998
in else
1.7999999999999998
in else
1.83
in else
1.8000000000000003
in else
1.83
in else
1.7699999999999998
in else
1.7200000000000002
in else
1.7
in else
1.67
in else
1.6199999999999999
in else
1.71
in else
1.7200000000000002
in else
1.7099999999999997
in else
1.7300000000000002
in else
1.7499999999999998
in else
1.76
in else
1.7400000000000002
in else
1.62
in else
1.58
in else
1.5499999999999998
in else
1.5
in else
1.5499999999999998
in else
1.46
in else
1.4700000000000002
in else
1.5
in else
1.5


2.81
in else
2.72
in else
2.7399999999999998
in else
2.72
in else
2.71
in else
2.74
in else
2.65
in else
2.75
in else
2.71
in else
2.7199999999999998
in else
2.69
in else
2.75
in else
2.73
in else
2.79
in else
2.76
in else
2.7399999999999998
in else
2.73
in else
2.77
in else
2.79
in else
2.75
in else
2.7600000000000002
in else
2.79
in else
2.77
in else
2.8200000000000003
in else
2.81
in else
2.7800000000000002
in else
2.73
in else
2.83
in else
2.83
in else
2.82
in else
2.84
in else
2.87
in else
2.85
in else
2.87
in else
2.91
in else
2.9000000000000004
in else
2.89
in else
2.84
in else
2.85
in else
2.79
in else
2.75
in else
2.77
in else
2.7600000000000002
in else
2.7800000000000002
in else
2.8099999999999996
in else
2.7199999999999998
in else
2.75
in else
2.73
in else
2.7
in else
2.73
in else
2.75
in else
2.77
in else
2.79
in else
2.81
in else
2.8099999999999996
in else
2.83
in else
2.7800000000000002
in else
2.72
in else
2.76
in else
2.75
in else
2.7
in else
2.64
in else
2.65
in else
2

2.4
in else
2.42
in else
2.4
in else
2.4
in else
2.3600000000000003
in else
2.34
in else
2.32
in else
2.33
in else
2.2800000000000002
in else
2.37
in else
2.37
in else
2.37
in else
2.42
in else
2.42
in else
2.4
in else
2.36
in else
2.3200000000000003
in else
2.29
in else
2.3200000000000003
in else
2.3200000000000003
in else
2.3099999999999996
in else
2.34
in else
2.3
in else
2.2700000000000005
in else
2.2800000000000002
in else
2.26
in else
2.2399999999999998
in else
2.28
in else
2.29
in else
2.33
in else
2.3499999999999996
in else
2.34
in else
2.31
in else
2.3
in else
2.29
in else
2.34
in else
2.28
in else
2.26
in else
2.28
in else
2.25
in else
2.26
in else
2.3
in else
2.31
in else
2.28
in else
2.27
in else
2.27
in else
2.25
in else
2.2600000000000002
in else
2.27
in else
2.25
in else
2.2199999999999998
in else
2.18
in else
2.1999999999999997
in else
2.1799999999999997
in else
2.21
in else
2.21
in else
2.22
in else
2.25
in else
2.2199999999999998
in else
2.15
in else
2.12
in else
2.14

0.8299999999999998
in thirty
0.8
in thirty
0.8
in thirty
0.8099999999999998
in thirty
0.8099999999999998
in thirty
0.78
in thirty
0.8300000000000001
in thirty
0.8800000000000001
in thirty
0.8900000000000001
in thirty
0.9300000000000002
in thirty
0.9400000000000002
in thirty
0.9200000000000002
in thirty
0.9700000000000002
in else
0.9900000000000002
in else
0.98
in else
1.0
in else
0.98
in else
0.98
in else
0.98
in else
0.95
in thirty
0.9099999999999999
in thirty
0.8999999999999999
in thirty
0.8999999999999999
in thirty
0.8800000000000001
in thirty
0.8899999999999997
in thirty
0.9300000000000002
in thirty
0.9299999999999999
in thirty
0.9599999999999997
in else
0.9599999999999997
in else
0.9599999999999997
in else
0.9199999999999997
in thirty
0.9099999999999999
in thirty
0.9000000000000001
in thirty
0.9099999999999999
in thirty
0.8999999999999997
in thirty
0.9299999999999999
in thirty
0.9100000000000001
in thirty
0.8700000000000001
in thirty
0.8899999999999999
in thirty
0.8900000000000001

0.2400000000000002
in ten
0.29000000000000004
in twenty
0.2400000000000002
in ten
0.23999999999999977
in ten
0.25
in ten
0.21999999999999997
in ten
0.25
in ten
0.27
in twenty
0.24999999999999978
in ten
0.22999999999999976
in ten
0.19999999999999996
in ten
0.2899999999999998
in twenty
0.2899999999999998
in twenty
0.2999999999999998
in twenty
0.30000000000000004
in twenty
0.29000000000000004
in twenty
0.2799999999999998
in twenty
0.2699999999999998
in twenty
0.25
in ten
0.24999999999999978
in ten
0.20999999999999996
in ten
0.18999999999999995
in ten
0.16999999999999993
in five
0.16999999999999993
in five
0.1499999999999999
in five
0.24999999999999978
in ten
0.2799999999999998
in twenty
0.28
in twenty
0.2599999999999998
in ten
0.2599999999999998
in ten
0.22999999999999998
in ten
0.27
in twenty
0.24999999999999978
in ten
0.24999999999999978
in ten
0.25
in ten
0.21999999999999975
in ten
0.21999999999999997
in ten
0.21999999999999997
in ten
0.20999999999999996
in ten
0.20999999999999996
in t

(3187,
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,

In [56]:
sum(np.array(conversion_rate) == 1)

2228